In [1]:
# Prepare environment
import os, sys
sys.path.insert(0, os.path.abspath('..'))
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Example starts here
------

In [2]:
import asyncio
from ibstract import IB
from ibstract import MarketDataBlock
from ibstract import HistDataReq
from ibstract.utils import dtest, dtcst, dtutc

### Instantiate IB and connect to Gateway/TWS
**When connecting to IB Gateway/TWS, an internal semaphore is acquired for up to 32 clients operating concurrently.**

In [3]:
ib = IB('127.0.0.1', 4002)
ib2 = IB()
ib2.connect('127.0.0.1', 4002)
print('ib connected? ', ib.connected)
print('ib2 connected? ', ib2.connected)
ib
ib2

ib connected?  True
ib2 connected?  True


<IB connected to 127.0.0.1:4002 clientId=21>

<IB connected to 127.0.0.1:4002 clientId=22>

### HistDataReq - A request object for historical data
** HistDataReq constructor signature: **
HistDataReq(sectype, symbol, barsize, timedur, timeend=None, datatype='TRADES', exchange='SMART', currency='USD')

In [4]:
req_list = [
    HistDataReq('Stock', 'GS', '1 hour', '5 d', dtest(2017,9,12,12)),
    HistDataReq('Stock', 'BAC', '1 day', '10 d'),  # timeend=None means datetime.now(tz=pytz.utc)
    HistDataReq('Stock', 'AMZN', '1 hour', '3 d', dtest(2017, 9, 12)),
    HistDataReq('Stock', 'GOOG', '30 mins', '3 d', dtutc(2017, 9, 12, 23)),
    HistDataReq('Stock', 'FB', '10 mins', '5 d', dtest(2017, 9, 12, 16)),
    HistDataReq('Stock', 'TVIX', '5 mins', '5 d', dtcst(2017, 9, 12, 16)),
]

for req in req_list:
    req

HistDataReq(Stock, GS, 1h, 5d, 2017-09-12 12:00:00-04:00, TRADES, SMART, USD)

HistDataReq(Stock, BAC, 1d, 10d, 2017-09-30 17:06:38.626811+00:00, TRADES, SMART, USD)

HistDataReq(Stock, AMZN, 1h, 3d, 2017-09-12 00:00:00-04:00, TRADES, SMART, USD)

HistDataReq(Stock, GOOG, 30m, 3d, 2017-09-12 23:00:00+00:00, TRADES, SMART, USD)

HistDataReq(Stock, FB, 10m, 5d, 2017-09-12 16:00:00-04:00, TRADES, SMART, USD)

HistDataReq(Stock, TVIX, 5m, 5d, 2017-09-12 16:00:00-05:00, TRADES, SMART, USD)

### Download data from IB requested by a HistDataReq
** Request contract details for a HistDataReq by the coroutine IB.hist_data_req_contract_details(). **

In [5]:
req = HistDataReq('Stock', 'GS', '1 hour', '5 d', dtest(2017,9,12,12))
loop = asyncio.get_event_loop()
contract_details_list = loop.run_until_complete(ib.hist_data_req_contract_details(req))
contract_details_list

[ContractDetails(summary=Contract(conId=4627828, symbol='GS', secType='STK', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='GS', tradingClass='GS'), marketName='GS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,RTHIGNOPG,SCALE,SCALEODD,SCALERST,SMARTSTG,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,ISE,CHX,ARCA,ISLAND,VWAP,DRCTEDGE,NSX,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,CVGXALGO,TPLUS1,PSX', priceMagnifier=1, longName='GOLDMAN SACHS GROUP INC', industry='Financial', category='Banks', subcategory='Diversified Banking Inst', timeZoneId='EST5EDT', tradingHours='20170930:CLOSED;20171001:CLOSED;20171002:0400-2000;20171003:0400-2000;20171004:0400-

** The coroutine IB.hist_data_req_timezone() downloads the exchange time zone for the requested security. **

In [6]:
req = HistDataReq('Stock', 'GS', '1 hour', '5 d', dtest(2017,9,12,12))
loop = asyncio.get_event_loop()
xchg_tz = loop.run_until_complete(ib.hist_data_req_timezone(req))
xchg_tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

** The coroutine IB.get_hist_data_async(*reqs) downloads historical data for one or multiple HistDataReqs concurrently, and output a list of MarketDataBlock instances in the order of input requests.**

In [7]:
loop = asyncio.get_event_loop()
blk_list = loop.run_until_complete(ib.req_hist_data_async(*req_list[:3]))
for blk in blk_list:
    blk.df.head()

opening    high     low  \
Symbol DataType BarSize TickerTime                                           
GS     TRADES   1h      2017-09-06 08:00:00-04:00   218.97  219.29  218.97   
                        2017-09-06 09:00:00-04:00   219.10  220.78  218.67   
                        2017-09-06 10:00:00-04:00   219.61  221.02  219.54   
                        2017-09-06 11:00:00-04:00   219.98  220.20  217.73   
                        2017-09-06 12:00:00-04:00   218.20  219.83  217.61   

                                                   closing  volume  barcount  \
Symbol DataType BarSize TickerTime                                             
GS     TRADES   1h      2017-09-06 08:00:00-04:00   219.20      28        11   
                        2017-09-06 09:00:00-04:00   219.58    4729      2596   
                        2017-09-06 10:00:00-04:00   220.01    4451      2722   
                        2017-09-06 11:00:00-04:00   218.20    4222      2500   
                        2017-09-06 12:00:00-04:00   219.80    2680      1809   

                                                   average  
Symbol DataType BarSize TickerTime                          
GS     TRADES   1h      2017-09-06 08:00:00-04:00  219.159  
                        2017-09-06 09:00:00-04:00  219.796  
                        2017-09-06 10:00:00-04:00  220.490  
                        2017-09-06 11:00:00-04:00  219.020  
                        2017-09-06 12:00:00-04:00  218.335

opening   high    low  \
Symbol DataType BarSize TickerTime                                         
BAC    TRADES   1d      2017-09-17 20:00:00-04:00    24.42  24.77  24.41   
                        2017-09-18 20:00:00-04:00    24.70  24.99  24.62   
                        2017-09-19 20:00:00-04:00    24.88  25.24  24.67   
                        2017-09-20 20:00:00-04:00    25.03  25.28  24.92   
                        2017-09-21 20:00:00-04:00    25.01  25.07  24.85   

                                                   closing  volume  barcount  \
Symbol DataType BarSize TickerTime                                             
BAC    TRADES   1d      2017-09-17 20:00:00-04:00    24.70  579550    110935   
                        2017-09-18 20:00:00-04:00    24.86  485710    100294   
                        2017-09-19 20:00:00-04:00    25.06  734771    138090   
                        2017-09-20 20:00:00-04:00    25.16  511256    104320   
                        2017-09-21 20:00:00-04:00    25.02  522287     97246   

                                                   average  
Symbol DataType BarSize TickerTime                          
BAC    TRADES   1d      2017-09-17 20:00:00-04:00  24.6995  
                        2017-09-18 20:00:00-04:00  24.8575  
                        2017-09-19 20:00:00-04:00  24.9735  
                        2017-09-20 20:00:00-04:00  25.1420  
                        2017-09-21 20:00:00-04:00  24.9845

opening    high     low  \
Symbol DataType BarSize TickerTime                                           
AMZN   TRADES   1h      2017-09-07 07:00:00-04:00   970.00  970.00  970.00   
                        2017-09-07 08:00:00-04:00   970.87  975.00  970.78   
                        2017-09-07 09:00:00-04:00   974.47  979.00  972.55   
                        2017-09-07 10:00:00-04:00   975.91  979.75  973.52   
                        2017-09-07 11:00:00-04:00   979.05  980.59  976.73   

                                                   closing  volume  barcount  \
Symbol DataType BarSize TickerTime                                             
AMZN   TRADES   1h      2017-09-07 07:00:00-04:00   970.00       2         2   
                        2017-09-07 08:00:00-04:00   975.00      49        36   
                        2017-09-07 09:00:00-04:00   975.72    3779      2276   
                        2017-09-07 10:00:00-04:00   979.01    3280      2159   
                        2017-09-07 11:00:00-04:00   979.42    2238      1509   

                                                   average  
Symbol DataType BarSize TickerTime                          
AMZN   TRADES   1h      2017-09-07 07:00:00-04:00  970.000  
                        2017-09-07 08:00:00-04:00  972.557  
                        2017-09-07 09:00:00-04:00  975.865  
                        2017-09-07 10:00:00-04:00  976.938  
                        2017-09-07 11:00:00-04:00  979.118

#### A blocking function IB.get_hist_data() is also available without using asyncio event loop. It drives the coroutine version internally.

In [8]:
req = HistDataReq('Stock', 'GS', '1 hour', '5 d', dtest(2017,9,12,12))
blk = ib.req_hist_data(req)[0]
blk

                                                   opening    high     low  \
Symbol DataType BarSize TickerTime                                           
GS     TRADES   1h      2017-09-06 08:00:00-04:00   218.97  219.29  218.97   
                        2017-09-06 09:00:00-04:00   219.10  220.78  218.67   
                        2017-09-06 10:00:00-04:00   219.61  221.02  219.54   
                        2017-09-06 11:00:00-04:00   219.98  220.20  217.73   
                        2017-09-06 12:00:00-04:00   218.20  219.83  217.61   
                        2017-09-06 13:00:00-04:00   219.77  220.50  219.41   
                        2017-09-06 14:00:00-04:00   219.61  219.80  218.90   
                        2017-09-06 15:00:00-04:00   219.33  219.70  218.85   
                        2017-09-06 16:00:00-04:00   218.83  219.09  218.83   
                        2017-09-06 17:00:00-04:00   218.98  218.98  218.98   
                        2017-09-06 18:00:00-04:00   218.70  218.